In [1]:
# SVM-based recommender system implemented from scratch using only NumPy and Pandas

import pandas as pd
import numpy as np

# Load cleaned data
df = pd.read_csv("cleaned_data.csv")

# Extract titles and features
titles = df["title"].reset_index(drop=True)
X = df.drop(columns=["title", "imdb_id"]).values.astype(float)
title_to_index = {title: i for i, title in enumerate(titles)}

# Standardize features manually
def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    std[std == 0] = 1
    return (X - mean) / std

X_std = standardize(X)

# Linear SVM classifier from scratch (using gradient descent)
class LinearSVM:
    def __init__(self, learning_rate=0.01, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        y_ = np.where(y <= 0, -1, 1)

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                if condition:
                    dw = self.lambda_param * self.w
                    db = 0
                else:
                    dw = self.lambda_param * self.w - y_[idx] * x_i
                    db = -y_[idx]
                self.w -= self.lr * dw
                self.b -= self.lr * db

    def decision_function(self, X):
        return np.dot(X, self.w) + self.b

# SVM-based recommendation function
def recommend_svm_from_scratch(input_title, top_n=10):
    if input_title not in title_to_index:
        return f"'{input_title}' not found in dataset."

    idx = title_to_index[input_title]

    y = np.zeros(X_std.shape[0])
    y[idx] = 1

    svm = LinearSVM()
    svm.fit(X_std, y)

    scores = svm.decision_function(X_std)
    scores[idx] = -np.inf  # Exclude input movie

    recommended_indices = np.argsort(scores)[::-1][:top_n]
    return titles.iloc[recommended_indices].tolist()

# Example usage
if __name__ == "__main__":
    input_title = input("Enter a movie title: ")
    recommendations = recommend_svm_from_scratch(input_title)

    print("\nTop 10 movie recommendations (SVM from scratch):")
    for i, title in enumerate(recommendations, 1):
        print(f"{i}. {title}")



Top 10 movie recommendations (SVM from scratch):
1. BPM (Beats per Minute)
2. Predator 2
3. Overboard
4. Charlie's Angels: Full Throttle
5. Heat
6. Big Trouble in Little China
7. The Devil's Rejects
8. Transformers: Revenge of the Fallen
9. Guardians of the Galaxy
10. Whisper of the Heart
